### Shortest Path with Alternating Colors

Consider a directed graph, with nodes labelled 0, 1, ..., n-1.  In this graph, each edge is either red or blue, and there could be self-edges or parallel edges.

Each [i, j] in red_edges denotes a red directed edge from node i to node j.  Similarly, each [i, j] in blue_edges denotes a blue directed edge from node i to node j.

Return an array answer of length n, where each answer[X] is the length of the shortest path from node 0 to node X such that the edge colors alternate along the path (or -1 if such a path doesn't exist).

 

In [5]:
from collections import defaultdict, deque
class Solution:
    def shortestAlternatingPaths(self, n: int, red_edges, blue_edges):
        graph_red = defaultdict(list)
        graph_blue = defaultdict(list)
        
        for x, y in red_edges:
            graph_red[x].append(y)
        
        for x, y in blue_edges:
            graph_blue[x].append(y)
            
        queue = deque()
        queue.append((0, 'R'))
        queue.append((0, 'B'))
        seen = {(0, 'R'), (0, 'B')}
        ans = [float('inf')]*n
        
        dist = 0
        while queue:
            size = len(queue)
            for _ in range(size):
                node, last_color = queue.popleft()
                ans[node] = min(dist, ans[node])
                
                new_color = 'B' if 'B' != last_color else 'R'
                graph = graph_blue if new_color == 'B' else graph_red
                
                for new_node in graph[node]:
                    if (new_node, new_color) not in seen:
                        seen.add((new_node, new_color))
                        queue.append((new_node, new_color))
            
            dist += 1
        
        for i in range(len(ans)):
            if ans[i] == float('inf'):
                ans[i] = -1
            
        return ans
                
                
Solution().shortestAlternatingPaths(n = 3, red_edges = [[0,1],[0,2]], blue_edges = [[1,0]])

[0, 1, 1]

### Minimum Number of Vertices to Reach All Nodes

Given a directed acyclic graph, with n vertices numbered from 0 to n-1, and an array edges where edges[i] = [fromi, toi] represents a directed edge from node fromi to node toi.

Find the smallest set of vertices from which all nodes in the graph are reachable. It's guaranteed that a unique solution exists.

Notice that you can return the vertices in any order.

In [7]:
from collections import Counter
def findSmallestSetOfVertices(n: int, edges):
    indegree = Counter()
    for x, y in edges:
        indegree[y] += 1

    return [node for node in range(n) if indegree[node] == 0]

findSmallestSetOfVertices(n = 6, edges = [[0,1],[0,2],[2,5],[3,4],[4,2]])

[0, 3]

### Detect Cycles in 2D Grid

Given a 2D array of characters grid of size m x n, you need to find if there exists any cycle consisting of the same value in grid.

A cycle is a path of length 4 or more in the grid that starts and ends at the same cell. From a given cell, you can move to one of the cells adjacent to it - in one of the four directions (up, down, left, or right), if it has the same value of the current cell.

Also, you cannot move to the cell that you visited in your last move. For example, the cycle (1, 1) -> (1, 2) -> (1, 1) is invalid because from (1, 2) we visited (1, 1) which was the last visited cell.

Return true if any cycle of the same value exists in grid, otherwise, return false.

In [8]:
class Solution:
    def containsCycle(self, grid) -> bool:
        state = {}
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if (i,j) not in state and self.hasCycle(i, j, grid, state, -1, -1):
                    return True
        
        return False
    
    def hasCycle(self, i, j, grid, state, pi, pj):
        state[(i,j)] = 'visiting'
        
        for x, y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
            if 0<=x<len(grid) and 0<=y<len(grid[0]) and (x,y) != (pi,pj) and grid[x][y] == grid[i][j]:
                
                if (x,y) not in state and self.hasCycle(x, y, grid, state, i, j):
                    return True
                
                if state[(x,y)] == 'visiting':
                    return True
        
        state[(i,j)] = 'visited'
        return False
                
grid = [["c","c","c","a"],["c","d","c","c"],["c","c","e","c"],["f","c","c","c"]]
Solution().containsCycle(grid)

True

### Find Latest Group of Size M

Given an array arr that represents a permutation of numbers from 1 to n. You have a binary string of size n that initially has all its bits set to zero.

At each step i (assuming both the binary string and arr are 1-indexed) from 1 to n, the bit at position arr[i] is set to 1. You are given an integer m and you need to find the latest step at which there exists a group of ones of length m. A group of ones is a contiguous substring of 1s such that it cannot be extended in either direction.

Return the latest step at which there exists a group of ones of length exactly m. If no such group exists, return -1.

In [10]:
class Solution:
    def findLatestStep(self, arr, m: int) -> int:
        n = len(arr)
        uf = UnionFind(n)
        res = [0]*n
        ans = -1
        for i in range(n):
            step = i+1
            index = arr[i]-1
            res[index] = 1
            uf.size[index] = 1
            uf.count[1] += 1
            
            if index-1 >= 0 and res[index-1] == 1:
                uf.union(index-1, index)
            
            if index+1 < n and res[index+1] == 1:
                uf.union(index, index+1)
            
            if uf.count[m] > 0:
                ans = step
        
        return ans
            
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.size = [0]*n
        self.count = Counter()
        
    def union(self, x, y):
        rootx = self.find(x)
        rooty = self.find(y)
        
        if rootx == rooty:
            return False
        
        size_x = self.size[rootx]
        size_y = self.size[rooty]
        
        self.count[size_x] -= 1
        self.count[size_y] -= 1
        
        new_size = size_x + size_y
        
        self.parent[rooty] = rootx
        self.size[rootx] = new_size
        self.count[new_size] += 1
        
        return True
    
    def find(self, x):
        while x != self.parent[x]:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x

Solution().findLatestStep(arr = [3,5,1,2,4], m = 1)

4

### Largest Component Size by Common Factor

Given a non-empty array of unique positive integers A, consider the following graph:

* There are A.length nodes, labelled A[0] to A[A.length - 1];
* There is an edge between A[i] and A[j] if and only if A[i] and A[j] share a common factor greater than 1.

Return the size of the largest connected component in the graph.

In [12]:
import math
class Solution:
    def largestComponentSize(self, A) -> int:
        graph = defaultdict(list)
        uf = UnionFind(len(A))
        
        for i, num in enumerate(A):
            prime_factors = self.get_prime_factors(num)
            for factor in prime_factors:
                graph[factor].append(i)
        
        for factor in graph:
            indices = graph[factor]
            for i in range(len(indices)-1):
                uf.union(indices[i], indices[i+1])
        
        counter = Counter()
        for i in range(len(A)):
            root = uf.find(i)
            counter[root] += 1
        
        return max(counter.values())
        
    def get_prime_factors(self, n):
        for i in range(2, int(math.sqrt(n))+1):
            if n % i == 0:
                return self.get_prime_factors(n//i) | set([i])
        return {n}

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
    
    def union(self, x, y):
        rootx = self.find(x)
        rooty = self.find(y)
        if rootx == rooty:
            return False
        self.parent[rooty] = rootx
        return True
    
    def find(self, x):
        while x != self.parent[x]:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
        
Solution().largestComponentSize([2,3,6,7,4,12,21,39])

8

### Minimum Number of Days to Disconnect Island

Given a 2D grid consisting of 1s (land) and 0s (water).  An island is a maximal 4-directionally (horizontal or vertical) connected group of 1s.

The grid is said to be connected if we have exactly one island, otherwise is said disconnected.

In one day, we are allowed to change any single land cell (1) into a water cell (0).

Return the minimum number of days to disconnect the grid.

In [15]:
class Solution:
    def minDays(self, grid) -> int:
        num_cc = self.dfs(grid)
        
        if num_cc != 1:
            return 0
        
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == 1:
                    grid[i][j] = 0
                    num_cc = self.dfs(grid)
                    if num_cc != 1:
                        return 1
                    grid[i][j] = 1
        
        return 2
    
    def dfs(self, grid):
        seen = set()
        num_cc = 0
        
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == 1 and (i,j) not in seen:
                    num_cc += 1
                    self.helper(i, j, seen, grid)
        
        return num_cc
    
    def helper(self, i, j, seen, grid):
        seen.add((i,j))
        for x, y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
            if 0<=x<len(grid) and 0<=y<len(grid[0]) and (x, y) not in seen and grid[x][y] == 1:
                self.helper(x, y, seen, grid)
        

Solution().minDays(grid = [[0,1,1,0],[0,1,1,0],[0,0,0,0]])

2

### Remove Max Number of Edges to Keep Graph Fully Traversable

Alice and Bob have an undirected graph of n nodes and 3 types of edges:

* Type 1: Can be traversed by Alice only.
* Type 2: Can be traversed by Bob only.
* Type 3: Can by traversed by both Alice and Bob.

Given an array edges where edges[i] = [typei, ui, vi] represents a bidirectional edge of type typei between nodes ui and vi, find the maximum number of edges you can remove so that after removing the edges, the graph can still be fully traversed by both Alice and Bob. The graph is fully traversed by Alice and Bob if starting from any node, they can reach all other nodes.

Return the maximum number of edges you can remove, or return -1 if it's impossible for the graph to be fully traversed by Alice and Bob.

In [17]:
class Solution:
    def maxNumEdgesToRemove(self, n: int, edges) -> int:
        edges.sort(key=lambda x: -x[0])
        uf1 = UnionFind(n+1)
        uf2 = UnionFind(n+1)
        
        e1 = e2 = 0
        res = 0
        
        for type_, node1, node2 in edges:
            
            if type_ == 3:
                val1 = uf1.union(node1, node2) 
                val2 = uf2.union(node1, node2)
                res += val1 or val2
                e1 += val1
                e2 += val2
            
            if type_ == 1:
                val = uf1.union(node1, node2)
                res += val
                e1 += val
            
            else:
                val = uf2.union(node1, node2)
                res += val
                e2 += val
        
        if e1 == e2 == n-1:
            return len(edges) - res
        return -1
    
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
    
    def union(self, x, y):
        rootx = self.find(x)
        rooty = self.find(y)
        
        if rootx == rooty:
            return False
        
        self.parent[rooty] = rootx
        return True
    
    def find(self, x):
        while x != self.parent[x]:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
        
Solution().maxNumEdgesToRemove(n = 4, edges = [[3,1,2],[3,2,3],[1,1,3],[1,2,4],[1,1,2],[2,3,4]])         

2

### Path With Minimum Effort

You are a hiker preparing for an upcoming hike. You are given heights, a 2D array of size rows x columns, where heights[row][col] represents the height of cell (row, col). You are situated in the top-left cell, (0, 0), and you hope to travel to the bottom-right cell, (rows-1, columns-1) (i.e., 0-indexed). You can move up, down, left, or right, and you wish to find a route that requires the minimum effort.

A route's effort is the maximum absolute difference in heights between two consecutive cells of the route.

Return the minimum effort required to travel from the top-left cell to the bottom-right cell.

In [4]:
from heapq import heappush, heappop
class Solution:
    def minimumEffortPath(self, heights) -> int:
        heap = []
        heappush(heap, (0, 0, 0))
        seen = set()
        
        while heap:
            dist, i, j = heappop(heap)
            if (i,j) in seen:
                continue
            
            seen.add((i,j))
            
            if i == len(heights)-1 and j == len(heights[0])-1:
                return dist
            
            curr_val = heights[i][j]
            
            for x, y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
                if 0<=x<len(heights) and 0<=y<len(heights[0]) and (x,y) not in seen:
                    new_dist = abs(curr_val - heights[x][y])
                    heappush(heap, (max(dist, new_dist), x, y))
        
Solution().minimumEffortPath(heights = [[1,2,2],[3,8,2],[5,3,5]])        

2

### Couples Holding Hands

N couples sit in 2N seats arranged in a row and want to hold hands. We want to know the minimum number of swaps so that every couple is sitting side by side. A swap consists of choosing any two people, then they stand up and switch seats.

The people and seats are represented by an integer from 0 to 2N-1, the couples are numbered in order, the first couple being (0, 1), the second couple being (2, 3), and so on with the last couple being (2N-2, 2N-1).

The couples' initial seating is given by row[i] being the value of the person who is initially sitting in the i-th seat.

In [3]:
from collections import defaultdict
class Solution:
    def minSwapsCouples(self, row) -> int:
        graph = defaultdict(list)
        for i in range(0, len(row), 2):
            couple_id1 = row[i]//2
            couple_id2 = row[i+1]//2
            graph[couple_id1].append(couple_id2)
            graph[couple_id2].append(couple_id1)
        
        res = 0
        seen = set()
        for couple_id in graph:
            if couple_id not in seen:
                cc = self.dfs(couple_id, graph, seen)
                res += cc-1
        
        return res
    
    def dfs(self, couple_id, graph, seen):
        seen.add(couple_id)
        cc = 1
        for neighbor in graph[couple_id]:
            if neighbor not in seen:
                cc += self.dfs(neighbor, graph, seen)
        
        return cc
                
            
Solution().minSwapsCouples(row = [0, 2, 1, 3])      

1

### Graph Connectivity With Threshold

We have n cities labeled from 1 to n. Two different cities with labels x and y are directly connected by a bidirectional road if and only if x and y share a common divisor strictly greater than some threshold. More formally, cities with labels x and y have a road between them if there exists an integer z such that all of the following are true:

* x % z == 0,
* y % z == 0, and
* z > threshold.

Given the two integers, n and threshold, and an array of queries, you must determine for each queries[i] = [ai, bi] if cities ai and bi are connected (i.e. there is some path between them).

Return an array answer, where answer.length == queries.length and answer[i] is true if for the ith query, there is a path between ai and bi, or answer[i] is false if there is no path.

In [6]:
class Solution:
    def areConnected(self, n: int, threshold: int, queries):
        uf = UnionFind(n+1)
        res = []
        
        for num in range(1, n+1):
            if num <= threshold:
                continue
            for neighbor in range(2*num, n+1, num):
                uf.union(num, neighbor)
        
        for x, y in queries:
            res.append(uf.find(x) == uf.find(y))
        
        return res
        

class UnionFind:
    
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0]*n
    
    def union(self, x, y):
        rootx = self.find(x)
        rooty = self.find(y)
        
        if rootx == rooty:
            return False
        
        if self.rank[rootx] > self.rank[rooty]:
            self.parent[rooty] = rootx
        else:
            self.parent[rootx] = rooty
            if self.rank[rootx] == self.parent[rooty]:
                self.rank[rooty] += 1
                
        return True
    
    def find(self, x):
        while x != self.parent[x]:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
        
Solution().areConnected(n = 6, threshold = 2, queries = [[1,4],[2,5],[3,6]])

[False, False, True]

### Lexicographically Smallest String After Applying Operations

You are given a string s of even length consisting of digits from 0 to 9, and two integers a and b.

You can apply either of the following two operations any number of times and in any order on s:

* Add a to all odd indices of s (0-indexed). Digits post 9 are cycled back to 0. For example, if s = "3456" and a = 5, s becomes "3951".
* Rotate s to the right by b positions. For example, if s = "3456" and b = 1, s becomes "6345".

Return the lexicographically smallest string you can obtain by applying the above operations any number of times on s.

In [8]:
class Solution:
    def findLexSmallestString(self, s: str, a: int, b: int) -> str:
        self.res = "9"*1000
        self.dfs(s, a, b, set())
        return self.res
    
    def dfs(self, s, a, b, seen):
        seen.add(s)
        self.res = min(self.res, s)
        
        cand1 = self.rotate(s, b)
        cand2 = self.add(s, a)
        
        if cand1 not in seen:
            self.dfs(cand1, a, b, seen)
        
        if cand2 not in seen:
            self.dfs(cand2, a, b, seen)
        
    def rotate(self, s, b):
        s = list(s)
        s = s[::-1]
        s[:b] = s[:b][::-1]
        s[b:] = s[b:][::-1]
        return ''.join(s)
    
    def add(self, s, a):
        string = ''
        for i, ch in enumerate(s):
            if i % 2 == 0:
                string += ch
            else:
                val = int(ch) + a
                string += str(val)[-1]
        return string
    
Solution().findLexSmallestString(s = "5525", a = 9, b = 2)

'2050'

### Frog Position After T Seconds

Given an undirected tree consisting of n vertices numbered from 1 to n. A frog starts jumping from the vertex 1. In one second, the frog jumps from its current vertex to another unvisited vertex if they are directly connected. The frog can not jump back to a visited vertex. In case the frog can jump to several vertices it jumps randomly to one of them with the same probability, otherwise, when the frog can not jump to any unvisited vertex it jumps forever on the same vertex. 

The edges of the undirected tree are given in the array edges, where edges[i] = [fromi, toi] means that exists an edge connecting directly the vertices fromi and toi.

Return the probability that after t seconds the frog is on the vertex target.

In [4]:
from collections import defaultdict
class Solution:
    def frogPosition(self, n: int, edges, t: int, target: int) -> float:
        adjlist = defaultdict(list)
        
        for x, y in edges:
            adjlist[x].append(y)
            adjlist[y].append(x)
            
        adjlist[1].append(0)
        
        return self.helper(1, adjlist, t, target, 0)
    
    def helper(self, curr_node, adjlist, t, target, parent):
        if t == 0:
            return int(curr_node == target)
        
        # reached a leaf node
        if len(adjlist[curr_node]) == 1:
            return int(curr_node == target)
        
        base_prob = 1 / (len(adjlist[curr_node]) - 1)
        
        for next_node in adjlist[curr_node]:
            if next_node == parent:
                continue  
            prob = base_prob * self.helper(next_node, adjlist, t-1, target, curr_node)
            
            if prob > 0:
                return prob
        
        return prob
            

Solution().frogPosition(n = 7, edges = [[1,2],[1,3],[1,7],[2,4],[2,6],[3,5]], t = 2, target = 4)

0.16666666666666666

### Minimum Moves to Move a Box to Their Target Location

Storekeeper is a game in which the player pushes boxes around in a warehouse trying to get them to target locations.

The game is represented by a grid of size m x n, where each element is a wall, floor, or a box.

Your task is move the box 'B' to the target position 'T' under the following rules:

* Player is represented by character 'S' and can move up, down, left, right in the grid if it is a floor (empy cell).
* Floor is represented by character '.' that means free cell to walk.
* Wall is represented by character '#' that means obstacle  (impossible to walk there). 
* There is only one box 'B' and one target cell 'T' in the grid.
* The box can be moved to an adjacent free cell by standing next to the box and then moving in the direction of the box. This is a push.
* The player cannot walk through the box.

Return the minimum number of pushes to move the box to the target. If there is no way to reach the target, return -1.

In [5]:
from collections import deque
class Solution:
    def minPushBox(self, grid) -> int:
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                val = grid[i][j]
                if val == 'S':
                    mx, my = i, j
                elif val == 'T':
                    tx, ty = i, j
                elif val == 'B':
                    bx, by = i, j
        
        queue = deque()
        queue.append((bx, by, mx, my, 0))
        seen = set()
        
        seen.add((bx, by, mx, my))
        
        while queue:
            bx, by, mx, my, count = queue.popleft()
            if bx == tx and by == ty:
                return count
            
            #left
            if self.isValid(bx, by-1, grid) and self.canReach(bx, by+1, mx, my, bx, by, grid, set()):
                if (bx, by-1, bx, by) not in seen:
                    queue.append((bx, by-1, bx, by, count+1))
                    seen.add((bx, by-1, bx, by))
            
            #up
            if self.isValid(bx-1, by, grid) and self.canReach(bx+1, by, mx, my, bx, by, grid, set()):
                if (bx-1, by, bx, by) not in seen:
                    queue.append((bx-1, by, bx, by, count+1))
                    seen.add((bx-1, by, bx, by))
            #right
            if self.isValid(bx, by+1, grid) and self.canReach(bx, by-1, mx, my, bx, by, grid, set()):
                if (bx, by+1, bx, by) not in seen:
                    queue.append((bx, by+1, bx, by, count+1))
                    seen.add((bx, by+1, bx, by))
            
            #down
            if self.isValid(bx+1, by, grid) and self.canReach(bx-1, by, mx, my, bx, by, grid, set()):
                if (bx+1, by, bx, by) not in seen:
                    queue.append((bx+1, by, bx, by, count+1))
                    seen.add((bx+1, by, bx, by))
        
        return -1
    
    def isValid(self, i, j, grid):
        return 0<=i<len(grid) and 0<=j<len(grid[0]) and grid[i][j] != '#'
    
    def canReach(self, tx, ty, mx, my, bx, by, grid, seen):
        queue = deque()
        queue.append((mx, my))
        seen.add((mx, my))
        
        while queue:
            mx, my = queue.popleft()
            if [mx, my] == [tx, ty]:
                return True
            
            for i, j in [(mx+1, my), (mx-1, my), (mx, my-1), (mx, my+1)]:
                if self.isValid(i, j, grid) and [i,j] != [bx, by] and (i,j) not in seen:
                    queue.append((i, j))
                    seen.add((i, j))
        
        return False
            
            
grid = [["#","#","#","#","#","#"],
       ["#","T","#","#","#","#"],
       ["#",".",".","B",".","#"],
       ["#",".","#","#",".","#"],
       ["#",".",".",".","S","#"],
       ["#","#","#","#","#","#"]]     

Solution().minPushBox(grid)

3

### Minimum Jumps to Reach Home

A certain bug's home is on the x-axis at position x. Help them get there from position 0.

The bug jumps according to the following rules:

* It can jump exactly a positions forward (to the right).
* It can jump exactly b positions backward (to the left).
* It cannot jump backward twice in a row.
* It cannot jump to any forbidden positions.
* The bug may jump forward beyond its home, but it cannot jump to positions numbered with negative integers.

Given an array of integers forbidden, where forbidden[i] means that the bug cannot jump to the position forbidden[i], and integers a, b, and x, return the minimum number of jumps needed for the bug to reach its home. If there is no possible sequence of jumps that lands the bug on position x, return -1.

In [2]:
from collections import deque
class Solution:
    def minimumJumps(self, forbidden, a: int, b: int, x: int) -> int:
        forb_set = set(forbidden)
        seen = set()
        queue = deque()
        queue.append((0, False))
        seen.add((0, False))
        maxPos = max(forbidden) + a + b + x
        
        ans = 0
        
        while queue:
            size = len(queue)
            for _ in range(size):
                pos, isBack = queue.popleft()
                
                if pos == x:
                    return ans
                
                if pos+a <= maxPos and (pos+a, False) not in seen and pos + a not in forb_set:
                    seen.add((pos+a, False))
                    queue.append((pos+a, False))
                
                if pos-b >= 0 and not isBack and (pos-b, True) not in seen and pos-b not in forb_set:
                    seen.add((pos-b, True))
                    queue.append((pos-b, True))
            
            ans += 1
        
        return -1
     
Solution().minimumJumps(forbidden = [14,4,18,1,15], a = 3, b = 15, x = 9)

3

### Maximal Network Rank

There is an infrastructure of n cities with some number of roads connecting these cities. Each roads[i] = [ai, bi] indicates that there is a bidirectional road between cities ai and bi.

The network rank of two different cities is defined as the total number of directly connected roads to either city. If a road is directly connected to both cities, it is only counted once.

The maximal network rank of the infrastructure is the maximum network rank of all pairs of different cities.

Given the integer n and the array roads, return the maximal network rank of the entire infrastructure.

In [4]:
from collections import defaultdict
class Solution:
    def maximalNetworkRank(self, n: int, roads) -> int:
        graph = defaultdict(set)
        for x, y in roads:
            graph[x].add(y)
            graph[y].add(x)
        
        res = -float('inf')
        for city1 in range(n):
            for city2 in range(city1+1, n):
                rank1 = len(graph[city1])
                rank2 = len(graph[city2]) - (1 if city1 in graph[city2] else 0)
                total_rank = rank1 + rank2
                res = max(res, total_rank)
        
        return res
 
Solution().maximalNetworkRank(n = 8, roads = [[0,1],[1,2],[2,3],[2,4],[5,6],[5,7]])

5

### The Earliest Moment When Everyone Become Friends

In a social group, there are N people, with unique integer ids from 0 to N-1.

We have a list of logs, where each logs[i] = [timestamp, id_A, id_B] contains a non-negative integer timestamp, and the ids of two different people.

Each log represents the time in which two different people became friends.  Friendship is symmetric: if A is friends with B, then B is friends with A.

Let's say that person A is acquainted with person B if A is friends with B, or A is a friend of someone acquainted with B.

Return the earliest time for which every person became acquainted with every other person. Return -1 if there is no such earliest time.

 

In [5]:
class Solution:
    def earliestAcq(self, logs, N: int) -> int:
        logs.sort()
        uf = UnionFind(N)
        edges = 0
        
        for ts, x, y in logs:
            if uf.union(x, y):
                edges += 1
                if edges == N - 1:
                    return ts
        return -1
    
class UnionFind:
    
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
    
    def union(self, x, y):
        rootx = self.find(x)
        rooty = self.find(y)
        
        if rootx == rooty:
            return False
        
        if self.rank[rootx] > self.rank[rooty]:
            self.parent[rooty] = rootx
        
        else:
            self.parent[rootx] = rooty
            if self.rank[rootx] == self.rank[rooty]:
                self.rank[rooty] += 1
        
        return True
    
    def find(self, x):
        while x != self.parent[x]:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
    
    
Solution().earliestAcq(logs = [[20190101,0,1],[20190104,3,4],[20190107,2,3],
                               [20190211,1,5],[20190224,2,4],[20190301,0,3],[20190312,1,2],[20190322,4,5]], N = 6)       

20190301

### Min Cost to Connect All Points

You are given an array points representing integer coordinates of some points on a 2D-plane, where points[i] = [xi, yi].

The cost of connecting two points [xi, yi] and [xj, yj] is the manhattan distance between them: |xi - xj| + |yi - yj|, where |val| denotes the absolute value of val.

Return the minimum cost to make all points connected. All points are connected if there is exactly one simple path between any two points.

In [9]:
from heapq import heapify, heappop
class Solution:
    def minCostConnectPoints(self, points) -> int:
        heap = []
        for i in range(len(points)):
            for j in range(i+1, len(points)):
                x1, y1 = points[i]
                x2, y2 = points[j]
            
                dist = abs(x1-x2) + abs(y1-y2)
                heap.append((dist, i, j))
        
        heapify(heap)
        uf = UnionFind(len(points))
        res = 0
        num_edges = 0
        
        while heap:
            dist, x, y = heappop(heap)
            if uf.union(x, y):
                res += dist
                num_edges += 1
            if num_edges == len(points)-1:
                break
                
        return res
    
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0]*n
        
    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        
        if root_x == root_y:
            return False
        
        if self.rank[root_x] > self.rank[root_y]:
            self.parent[root_y] = root_x
        
        else:
            self.parent[root_x] = root_y
            if self.rank[root_x] == self.rank[root_y]:
                self.rank[root_y] += 1
        
        return True
    
    def find(self, x):
        while x != self.parent[x]:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
                
        
Solution().minCostConnectPoints(points = [[3,12],[-2,5],[-4,1]])      

18

### Throne Inheritance

A kingdom consists of a king, his children, his grandchildren, and so on. Every once in a while, someone in the family dies or a child is born.

The kingdom has a well-defined order of inheritance that consists of the king as the first member.

Let's define the recursive function Successor(x, curOrder), which given a person x and the inheritance order so far, returns who should be the next person after x in the order of inheritance.

In [3]:
class ThroneInheritance:

    def __init__(self, kingName: str):
        self.children = defaultdict(list)
        self.dead = set()
        self.king = kingName
        
    def birth(self, parentName: str, childName: str) -> None:
        self.children[parentName].append(childName)
        
    def death(self, name: str) -> None:
        self.dead.add(name)
        
    def getInheritanceOrder(self):
        self.order = []
        self.helper(self.king)
        return self.order
    
    def helper(self, king):
        if king not in self.dead:
            self.order.append(king)
        for child in self.children[king]:
            self.helper(child)


# Your ThroneInheritance object will be instantiated and called as such:
# obj = ThroneInheritance(kingName)
# obj.birth(parentName,childName)
# obj.death(name)
# param_3 = obj.getInheritanceOrder()

### Checking Existence of Edge Length Limited Paths

An undirected graph of n nodes is defined by edgeList, where edgeList[i] = [ui, vi, disi] denotes an edge between nodes ui and vi with distance disi. Note that there may be multiple edges between two nodes.

Given an array queries, where queries[j] = [pj, qj, limitj], your task is to determine for each queries[j] whether there is a path between pj and qj such that each edge on the path has a distance strictly less than limitj .

Return a boolean array answer, where answer.length == queries.length and the jth value of answer is true if there is a path for queries[j] is true, and false otherwise.

In [1]:
class Solution:
    def distanceLimitedPathsExist(self, n: int, edgeList, queries):
        q = []
        for i in range(len(queries)):
            x, y, limit = queries[i]
            q.append((i, x, y, limit))
        queries = q
        
        queries.sort(key = lambda x : x[-1])
        edgeList.sort(key = lambda x : x[-1])
        uf = UnionFind(n)
        
        i = 0; j = 0
        ans = [False]*len(queries)
        
        while i < len(queries):
            limit = queries[i][-1]
            index = queries[i][0]
            while j < len(edgeList) and edgeList[j][-1] < limit:
                x, y, dist = edgeList[j]
                uf.union(x, y)
                j += 1
                
            ans[index] = uf.isConnected(queries[i][1], queries[i][2])
            i += 1
        
        return ans
            
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0]*n
    
    def isConnected(self, x, y):
        return self.find(x) == self.find(y)
        
    def union(self, x, y):
        rootx = self.find(x)
        rooty = self.find(y)
        
        if rootx == rooty:
            return False
        
        if self.rank[rootx] > self.rank[rooty]:
            self.parent[rooty] = rootx
        
        else:
            self.parent[rootx] = rooty
            if self.rank[rootx] == self.rank[rooty]:
                self.rank[rooty] += 1
        
        return True
    
    def find(self, x):
        while x != self.parent[x]:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x

Solution().distanceLimitedPathsExist(n = 3, edgeList = [[0,1,2],[1,2,4],[2,0,8],[1,0,16]], queries = [[0,1,2],[0,2,5]])

[False, True]